# Ocean Heat Content difference between the 1.1x run and the base run 

CMFW May 2025

McFLURRIE project, ECCO Summer School, Pacific Grove CA

Notebook creates gifs and frames of the difference between the base run and an increased-runoff ocean heat content in the upper 20m of ECCOv4r5 daily output. These plots are in the Arctic region.

In [1]:
# -------------------
# Import Packages
# -------------------

# tell Python to use the ecco_v4_py in the 'ECCOv4-py' repository
from os.path import join,expanduser
import sys
# identify user's home directory
user_home_dir = expanduser('~')
# import the ECCOv4 py library 
sys.path.insert(0,join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco

# Generic Packages
import os
import glob
import cmocean
import cmocean
from dask.distributed import Client
import datetime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import zarr

In [1]:
# --------------------------------------------------------------
# Function to convert from iteration number (ECCO) to datetime
# Created by Mike Wood
# --------------------------------------------------------------

from datetime import timedelta, datetime

def date_to_iter_number(date,seconds_per_iter = 3600):
    total_seconds = (date-datetime(1992,1,1)).total_seconds()
    iter_number = total_seconds/seconds_per_iter
    return(iter_number)

def iter_number_to_date(iter_number,seconds_per_iter=3600):
    total_seconds = iter_number*seconds_per_iter
    date = datetime(1992,1,1) + timedelta(seconds=total_seconds)
    return(date)

In [4]:
# --------------------------------------
# Begin a dask client for ease + speed
# --------------------------------------

from dask.distributed import Client

#  connect to existing LocalCluster
# the port number will be different!
client = Client("tcp://127.0.0.1:34853")
client.ncores
client.restart()

In [5]:
# ---------------------------
# Import the model geometry
# ---------------------------

# Your path may vary!
geom = xr.open_dataset('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')

In [69]:
# ---------------------------------------------------
# Check dates -- can change to desired year 
# ---------------------------------------------------

oct_2019_start = date_to_iter_number(datetime(2019,10,1))
dec_2019_end = date_to_iter_number(datetime(2019,12,31))

In [70]:
# ----------------------------------------------------
# Load in temperature files for your year -- base run
# ----------------------------------------------------

# Choose whichever time span is interesting (be wary of memory!)
# I chose October, November and December 2019

# Create path from which we pull data
input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/'
pattern = os.path.join(input_dir, 'THETA_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your time range
start_num = oct_2014_start
end_num = dec_2014_end

# Initialise empty list for storage
theta_DA_list = []

# Load in each file
for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            theta_test = ecco.read_llc_to_tiles(input_dir, filename);
            theta_test = np.where(geom.hFacC == 1, theta_test, np.nan);

            tile = range(0, 13)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            theta_DA = xr.DataArray(
                theta_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            theta_DA_list.append(theta_DA);

# Concatenate all files
theta_OND2014_base = xr.concat(theta_DA_list, dim='time');

load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/THETA_daily_mean.0000199416.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/THETA_daily_mean.0000199440.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/THETA_daily_mean.0000199464.data
load_binary_array: 

In [71]:
# ----------------------------------------------------
# Load in temperature files for your year -- 1.1x run
# ----------------------------------------------------

input_dir = '/efs_ecco/cwilliam/tenpercent/diags/THETA_daily_mean/'
pattern = os.path.join(input_dir, 'THETA_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_2014_start
end_num = dec_2014_end

theta_DA_list = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            theta_test = ecco.read_llc_to_tiles(input_dir, filename);
            theta_test = np.where(geom.hFacC == 1, theta_test, np.nan);

            tile = range(0, 13)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            theta_DA = xr.DataArray(
                theta_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            theta_DA_list.append(theta_DA);

# Concatenate all valid files
theta_OND2014_tenper = xr.concat(theta_DA_list, dim='time');

load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/THETA_daily_mean/THETA_daily_mean.0000199416.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/THETA_daily_mean/THETA_daily_mean.0000199440.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/THETA_daily_mean/THETA_daily_mean.0000199464.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4


In [72]:
# -------------------------------------------------------
# Define the specific heat capacity and density of water
# -------------------------------------------------------

# Value taken from: Josey and Sinha, 2022.
# Should check around to see if this is similar in other writing

c = 3850
rho = 1025

In [73]:
# -------------------------------
# Limit depth to top 15m (test out for 15m and see how it changes!)
# --------------------------------

theta_base = theta_OND2014_base.where(geom.Z>-20)
theta_tenper = theta_OND2014_tenper.where(geom.Z>-20)
rA = geom.rA.where(geom.Z>-20)
drF = geom.drF.where(geom.Z>-20)

In [74]:
# ----------------------------------------
# Calculating the mcT part, and integrate
# ----------------------------------------

cellvol = rA*drF
T_ref = np.zeros_like(theta_base) - 2

H_cell_base = (rho*cellvol) * c * (theta_base - T_ref)
H_xy_base = (H_cell_base).sum('k')
H_xy_base = H_xy_base.where(H_xy_base != 0)

H_cell_tenper = (rho*cellvol) * c * (theta_tenper - T_ref)
H_xy_tenper = (H_cell_tenper).sum('k')
H_xy_tenper = H_xy_tenper.where(H_xy_base != 0)

In [75]:
# ---------------------
# Plotting the gif
# ---------------------

import imageio
from matplotlib.colors import LogNorm

# Assume your DataArray is: (time, j, i)
# If it's 4D (e.g., time, k, j, i), select a level:
data = H_xy_tenper.isel(tile=6) - H_xy_base.isel(tile=6)

# Directory for temporary frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)
cmap = cmocean.cm.diff.copy()
cmap.set_bad(color='gray')

filenames = []

# Loop through time steps
for t in range(len(data.time)):  # or len(data.time) to use all
    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(data.isel(time=t)/1e15, origin='lower', cmap=cmap, vmin=-.75, vmax=.75)
    ax.set_title(str(data.time[t].values))  # Use time label
    fig.colorbar(im, ax=ax)
    
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = 'OHC_2014_tenper_base.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Cleanup
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

/tmp/ipykernel_59/3913135459.py:34: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved to OHC_2014_tenper_base.gif
